In [1]:
import os
import argparse
import json
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, ELU, Lambda, Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
# Display settings
%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [3]:
from model import *
df = load_data(['./data/sim_data/','./data/udacity_data/'])

In [4]:
from preprocess import *

In [5]:
df = shift_steering(df)